In [ ]:
import math
import torch
import time
import random
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.parameter import Parameter
from torch.nn import init
from torch import Tensor
from scipy.special import gamma 
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath, amssymb}'
custom_params = {
    # Font and text settings
    'font.size': 8,                
    'axes.labelsize': 8,           
    'axes.titlesize': 8,           
    'xtick.labelsize': 8,          
    'ytick.labelsize': 8,          
    'legend.fontsize': 6,          
    
    # Line properties
    'lines.linewidth': 1.2,        
    'lines.markersize': 4,         
    
    # Figure dimensions and quality
    'figure.figsize': (3.5, 2.5),  
    
    # Axis properties
    'axes.linewidth': 0.8,         
    'grid.linewidth': 0.5,         
    'grid.alpha': 0.3,             
}

# Apply the customized parameters to matplotlib
plt.rcParams.update(custom_params)


def MSE(pred,true):
    return np.mean((pred-true)**2)

def MAE(pred, true):
    return np.mean(np.abs(pred-true))

def RMSE(pred,true):
    return np.sqrt(np.mean((pred-true)**2))

def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def clip_matrix_norm(matrix, max_norm):
    norm = torch.norm(matrix)
    if norm > max_norm:
        matrix = matrix * (max_norm / norm)
    return matrix


class Fractional_Order_Matrix_Differential_Solver(torch.autograd.Function):
    @staticmethod
    def forward(ctx,input1,w,b,alpha,c):
        alpha = torch.tensor(alpha)
        c = torch.tensor(c)
        ctx.save_for_backward(input1,w,b,alpha,c)
        outputs = input1@w + b
        return outputs

    @staticmethod
    def backward(ctx, grad_outputs):
        input1,w,b,alpha,c = ctx.saved_tensors
        x_fractional, w_fractional = Fractional_Order_Matrix_Differential_Solver.Fractional_Order_Matrix_Differential_Linear(input1,w,b,alpha,c)   
        x_grad = grad_outputs@x_fractional
        w_grad = w_fractional@grad_outputs
        b_grad = grad_outputs.sum(dim=0)
        return x_grad, w_grad, b_grad,None,None
          
    @staticmethod
    def Fractional_Order_Matrix_Differential_Linear(xs,ws,b,alpha,c):
        wf = ws[:,0].view(1,-1)
        #main
        w_main = torch.mul(xs,(torch.abs(wf)+1e-8)**(1-alpha)/gamma(2-alpha))
        #partial
        w_partial = torch.mul((xs@wf.T).expand(xs.shape) - torch.mul(xs,wf) + b[0], torch.sgn(wf)*(torch.abs(wf)+1e-8)**(-alpha)/gamma(1-alpha))
        return ws.T, (w_main + clip_matrix_norm(w_partial,c)).transpose(-2,-1)

class FLinear(nn.Module):
    
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, alpha=0.9, c=1.0, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.c = c
        self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, x):
        return Fractional_Order_Matrix_Differential_Solver.apply(x, self.weight.T, self.bias, self.alpha,self.c)

    def extra_repr(self) -> str:
        return f"in_features={self.in_features}, out_features={self.out_features}, bias={self.bias is not None}"

In [10]:
# Configuration parameters
slide_windows_size = 192  # Input sequence length
pred_length = 384        # Prediction horizon length
stock = 'ETTh1'            # Dataset name (ETTm2 for comparison)
features_j = 6           # Target feature index (DJI:4, ETTm2:6)
num_feature = features_j + 1         #(DJI:5, ETTm2:7)

# Load data
root = r'C:\Users\Administrator\torch_zxj\博士第四篇代码'
df_DJIA = pd.read_csv(root+'/data/'+stock+'.csv')
# df_DJIA = pd.read_csv(r'./data/'+stock+'.csv')
del df_DJIA['date']  # Remove date column

# 1. Split data first (7:1:2 ratio)
def split_time_series(data, train_ratio=0.7, val_ratio=0.1):
    """
    Split time series data in chronological order
    
    Args:
        data: Complete time series data
        train_ratio: Proportion for training set
        val_ratio: Proportion for validation set
    
    Returns:
        train_data, val_data, test_data: Split datasets
    """
    n_samples = len(data)
    train_end = int(n_samples * train_ratio)
    val_end = train_end + int(n_samples * val_ratio)
    
    # Split in chronological order (important for time series)
    train_data = data[:train_end]
    val_data = data[train_end:val_end]
    test_data = data[val_end:]
    
    return train_data, val_data, test_data

# Split raw data first
train_raw, val_raw, test_raw = split_time_series(df_DJIA.values, 0.7, 0.1)

print(f"Data split results:")
print(f"  Training set: {len(train_raw)} samples ({len(train_raw)/len(df_DJIA)*100:.1f}%)")
print(f"  Validation set: {len(val_raw)} samples ({len(val_raw)/len(df_DJIA)*100:.1f}%)")
print(f"  Test set: {len(test_raw)} samples ({len(test_raw)/len(df_DJIA)*100:.1f}%)")

# scaler = MinMaxScaler()
scaler = StandardScaler()
scaler.fit(train_raw)

# Transform all datasets using training set statistics
train_scaled = scaler.transform(train_raw)
val_scaled = scaler.transform(val_raw)  # Use training set statistics
test_scaled = scaler.transform(test_raw)  # Use training set statistics

# 3. Create sequences for time series forecasting
def create_sequences(data, slide_windows_size, pred_length, target_idx):
    """
    Create input-output sequences for time series forecasting
    
    Args:
        data: Multivariate time series data
        slide_windows_size: Input sequence length (look-back window)
        pred_length: Output sequence length (forecast horizon)
        target_idx: Index of target feature to predict
    
    Returns:
        X: Input sequences [samples, seq_len, features]
        y: Target sequences [samples, pred_length]
    """
    X, y = [], []
    for i in range(len(data) - slide_windows_size - pred_length + 1):
        # Input sequence: sliding window of features
        X.append(data[i:i+slide_windows_size, :])  # [seq_len, features]
        # Target sequence: future values of target feature
        y.append(data[i+slide_windows_size:i+slide_windows_size+pred_length, target_idx])  
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

# Create sequences for each dataset
X_train, y_train = create_sequences(train_scaled, slide_windows_size, pred_length, features_j)
X_val, y_val = create_sequences(val_scaled, slide_windows_size, pred_length, features_j)
X_test, y_test = create_sequences(test_scaled, slide_windows_size, pred_length, features_j)

print(f"\nSequence creation results:")
print(f"  Training set: X{X_train.shape}, y{y_train.shape}")
print(f"  Validation set: X{X_val.shape}, y{y_val.shape}")
print(f"  Test set: X{X_test.shape}, y{y_test.shape}")

# 4. Convert to PyTorch tensors
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"\nUsing device: {device}")

X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_val_tensor = torch.FloatTensor(X_val).to(device)
y_val_tensor = torch.FloatTensor(y_val).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

print(f"\nTensor shapes:")
print(f"  X_train_tensor: {X_train_tensor.shape}")
print(f"  y_train_tensor: {y_train_tensor.shape}")
print(f"  X_val_tensor: {X_val_tensor.shape}")
print(f"  y_val_tensor: {y_val_tensor.shape}")
print(f"  X_test_tensor: {X_test_tensor.shape}")
print(f"  y_test_tensor: {y_test_tensor.shape}")

Data split results:
  Training set: 12194 samples (70.0%)
  Validation set: 1742 samples (10.0%)
  Test set: 3484 samples (20.0%)

Sequence creation results:
  Training set: X(11619, 192, 7), y(11619, 384)
  Validation set: X(1167, 192, 7), y(1167, 384)
  Test set: X(2909, 192, 7), y(2909, 384)

Using device: cuda:0

Tensor shapes:
  X_train_tensor: torch.Size([11619, 192, 7])
  y_train_tensor: torch.Size([11619, 384])
  X_val_tensor: torch.Size([1167, 192, 7])
  y_val_tensor: torch.Size([1167, 384])
  X_test_tensor: torch.Size([2909, 192, 7])
  y_test_tensor: torch.Size([2909, 384])


In [11]:
batch_size = 256
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [12]:
# momentums = [0.1,0.5,0.9]
weight_decays = [1e-3,1e-4,1e-5]
lrs = [3e-2,1e-2,3e-3,1e-3,3e-4]             

# root = r'C:\Users\Administrator\torch_zxj\博士第四篇代码'
# batch_size = 256
num_epochs = 200
set_seed()
# train_data = TensorDataset(X_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1=512, hidden_size2=256,output_size=pred_length):  
        super().__init__()
        self.flatten = nn.Flatten()
        # self.linear1 = FLinear(input_size, hidden_size1, alpha,c) 
        self.linear1 = nn.Linear(input_size, hidden_size1)    
        self.leakrelu1 = nn.LeakyReLU()                          
        # self.linear2 = FLinear(hidden_size1, hidden_size2, alpha,c) 
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)    
        self.leakrelu2 = nn.LeakyReLU()
        # self.linear3 = FLinear(hidden_size2, output_size, alpha,c)  
        self.linear3 = nn.Linear(hidden_size2, output_size)  
        
    def forward(self, x):
        x = self.flatten(x)    # (batch_size, seq_len*num_features)
        x = self.leakrelu1(self.linear1(x))
        x = self.leakrelu2(self.linear2(x))
        x = self.linear3(x)
        return x
best_lr = 0
# best_momentum = 0
best_weight_decay = 0
best_evaluation = 10000
for lr in lrs:
    # for momentum in momentums:
    for weight_decay in weight_decays:   
        print('')
        print('lr:',lr)
        # print('momentum:',momentum)
        print('weight_decay:',weight_decay)
        set_seed()
        model = MLP(input_size=slide_windows_size*num_feature).to(device)
        best_loss = 10000
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)   #adam
        # optimizer = torch.optim.SGD(model.parameters(),lr=lr)
        # optimizer = torch.optim.SGD(model.parameters(),lr=lr,momentum=momentum,weight_decay=weight_decay)  #  SGD
        for ii in range(num_epochs):
            model.train()
            loss_sum = 0
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss_sum += loss
                loss.backward()   #The default value of retain_graph is False.
                optimizer.step()
                
            model.eval()
            with torch.no_grad():
                Val_outputs = model(X_val_tensor)
                RMSE_val = RMSE(y_val_tensor.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
                MAE_val = MAE(y_val_tensor.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
                MAPE_val = MAPE(y_val_tensor.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
                best_val = RMSE_val + MAE_val+MAPE_val

                if best_loss > best_val:
                    best_loss = best_val
                    torch.save(model.state_dict(), root+'/model/table2-3/'+stock+'_model_fractional_adam'+'_'+str(weight_decay)+'_'+str(lr)+'_.pth')    #adam
                    # torch.save(model.state_dict(), root+'/model/table2-3/'+stock+'_model_fractional'+str(momentum)+'_'+str(weight_decay)+'_'+str(lr)+'_.pth')   #SGDM
                    # torch.save(model.state_dict(), r'./model/table2-3/'+stock+'_model_fractional_'+str(momentum)+'_'+str(weight_decay)+'_'+str(lr)+'_.pth') 

        model.load_state_dict(torch.load(root+'/model/table2-3/'+stock+'_model_fractional_adam'+'_'+str(weight_decay)+'_'+str(lr)+'_.pth'))   #adam
        # model.load_state_dict(torch.load('./model/table2-3/'+stock+'_model_fractional_'+str(momentum)+'_'+str(weight_decay)+'_'+str(lr)+'_.pth'))
        # model.load_state_dict(torch.load(root+'/model/table2-3/'+stock+'_model_fractional_'+str(momentum)+'_'+str(weight_decay)+'_'+str(lr)+'_.pth'))   #SGDM
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test_tensor)
        RMSE_test = RMSE(y_test_tensor.cpu().numpy(),test_outputs.cpu().detach().numpy())
        MAE_test = MAE(y_test_tensor.cpu().numpy(),test_outputs.cpu().detach().numpy())
        MAPE_test = MAPE(y_test_tensor.cpu().numpy(),test_outputs.cpu().detach().numpy())
        # print(str(momentum)+'_'+str(weight_decay)+'_'+str(lr)+'_'+f'RMSE:{RMSE_test:.6f}')
        # print(str(momentum)+'_'+str(weight_decay)+'_'+str(lr)+'_'+f'MAE:{MAE_test:.6f}')
        # print(str(momentum)+'_'+str(weight_decay)+'_'+str(lr)+'_'+f'MAPE:{MAPE_test:.6f}')
        print(str(lr)+'_'+str(weight_decay)+'_'+f'RMSE:{RMSE_test:.6f}')
        print(str(lr)+'_'+str(weight_decay)+'_'+f'MAE:{MAE_test:.6f}')
        print(str(lr)+'_'+str(weight_decay)+'_'+f'MAPE:{MAPE_test:.6f}')
        print('RMSE_test+MAE_test+MAPE_test:',RMSE_test+MAE_test+MAPE_test)
        if best_evaluation > RMSE_test + MAE_test + MAPE_test:
            best_evaluation = RMSE_test + MAE_test + MAPE_test
            print('Best evaluation:',best_evaluation)
            best_lr = lr
            # best_momentum = momentum
            best_weight_decay = weight_decay
print('best_lr:',best_lr)
# print('best_momentum:',best_momentum)
print('best_weight_decay:',best_weight_decay)
print('best_evaluation:',best_evaluation)


lr: 0.03
weight_decay: 0.001
0.03_0.001_RMSE:0.399395
0.03_0.001_MAE:0.320271
0.03_0.001_MAPE:0.683155
RMSE_test+MAE_test+MAPE_test: 1.4028218
Best evaluation: 1.4028218

lr: 0.03
weight_decay: 0.0001


C:\Users\Administrator\AppData\Local\Temp\ipykernel_31156\2194696802.py:64: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((pred - true) / true))


0.03_0.0001_RMSE:0.542679
0.03_0.0001_MAE:0.435128
0.03_0.0001_MAPE:1.498819
RMSE_test+MAE_test+MAPE_test: 2.476626

lr: 0.03
weight_decay: 1e-05
0.03_1e-05_RMSE:1.078251
0.03_1e-05_MAE:0.940961
0.03_1e-05_MAPE:0.481940
RMSE_test+MAE_test+MAPE_test: 2.501152

lr: 0.01
weight_decay: 0.001
0.01_0.001_RMSE:0.489277
0.01_0.001_MAE:0.396537
0.01_0.001_MAPE:1.468694
RMSE_test+MAE_test+MAPE_test: 2.3545077

lr: 0.01
weight_decay: 0.0001
0.01_0.0001_RMSE:0.519285
0.01_0.0001_MAE:0.399450
0.01_0.0001_MAPE:0.750689
RMSE_test+MAE_test+MAPE_test: 1.6694237

lr: 0.01
weight_decay: 1e-05
0.01_1e-05_RMSE:0.560407
0.01_1e-05_MAE:0.440164
0.01_1e-05_MAPE:2.125403
RMSE_test+MAE_test+MAPE_test: 3.1259742

lr: 0.003
weight_decay: 0.001
0.003_0.001_RMSE:0.509518
0.003_0.001_MAE:0.411164
0.003_0.001_MAPE:1.277599
RMSE_test+MAE_test+MAPE_test: 2.1982808

lr: 0.003
weight_decay: 0.0001
0.003_0.0001_RMSE:0.452174
0.003_0.0001_MAE:0.361757
0.003_0.0001_MAPE:0.592760
RMSE_test+MAE_test+MAPE_test: 1.4066907

lr: 

In [13]:
# momentum = 0.1
weight_decay = 0.001
lr = 0.03

alphas = [0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2]    # DJI:0.999,0.99,0.98,0.95,0.9,0.85,0.8,0.75  ETTh1:0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2
cs = [0.1,0.2,0.5,1.0,2.0,5.0,10.0]

# root = r'C:\Users\Administrator\torch_zxj\博士第四篇代码'
# batch_size = 256
num_epochs = 200
set_seed()
# train_data = TensorDataset(X_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1=512, hidden_size2=256,output_size=pred_length):  
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear1 = FLinear(input_size, hidden_size1, alpha,c) 
        # self.linear1 = nn.Linear(input_size, hidden_size1)    
        self.leakrelu1 = nn.LeakyReLU()                          
        self.linear2 = FLinear(hidden_size1, hidden_size2, alpha,c) 
        # self.linear2 = nn.Linear(hidden_size1, hidden_size2)    
        self.leakrelu2 = nn.LeakyReLU()
        self.linear3 = FLinear(hidden_size2, output_size, alpha,c)  
        # self.linear3 = nn.Linear(hidden_size2, output_size)  
        
    def forward(self, x):
        x = self.flatten(x)    # (batch_size, seq_len*num_features)
        x = self.leakrelu1(self.linear1(x))
        x = self.leakrelu2(self.linear2(x))
        x = self.linear3(x)
        return x
best_c = 0
best_alpha = 0
best_evaluation = 10000
for alpha in alphas:
    for c in cs:
        print('')
        print('alpha:',alpha)
        print('c:',c)
        set_seed()
        model = MLP(input_size=slide_windows_size*num_feature).to(device)
        best_loss = 10000
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
        # optimizer = torch.optim.SGD(model.parameters(),lr=lr)
        # optimizer = torch.optim.SGD(model.parameters(),lr=lr,momentum=momentum,weight_decay=weight_decay)  #sgdm
        for ii in range(num_epochs):
            model.train()
            loss_sum = 0
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss_sum += loss
                loss.backward()   #The default value of retain_graph is False.
                optimizer.step()
                
            model.eval()
            with torch.no_grad():
                Val_outputs = model(X_val_tensor)
                RMSE_val = RMSE(y_val_tensor.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
                MAE_val = MAE(y_val_tensor.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
                MAPE_val = MAPE(y_val_tensor.cpu().detach().numpy(),Val_outputs.cpu().detach().numpy())
                best_val = RMSE_val + MAE_val+MAPE_val

                if best_loss > best_val:
                    best_loss = best_val
                    torch.save(model.state_dict(), root+'/model/table2-3/'+stock+'_model_fractional_adam'+'_'+str(weight_decay)+'_'+str(lr)+'_'+str(alpha)+'_'+str(c)+'_.pth')  #Fadam
                    # torch.save(model.state_dict(), root+'/model/table2-3/'+stock+'_model_fractional_'+str(momentum)+'_'+str(weight_decay)+'_'+str(lr)+'_'+str(alpha)+'_'+str(c)+'_.pth')  #FSGDM 
                    # torch.save(model.state_dict(), r'./model/table2-3/'+stock+'_model_fractional_'+str(momentum)+'_'+str(weight_decay)+'_'+str(lr)+'_'+str(alpha)+'_'+str(c)+'_.pth') 

        # model.load_state_dict(torch.load('./model/table2-3/'+stock+'_model_fractional_'+str(momentum)+'_'+str(weight_decay)+'_'+str(lr)+'_'+str(alpha)+'_'+str(c)+'_.pth'))
        # model.load_state_dict(torch.load(root+'/model/table2-3/'+stock+'_model_fractional_'+str(momentum)+'_'+str(weight_decay)+'_'+str(lr)+'_'+str(alpha)+'_'+str(c)+'_.pth'))
        model.load_state_dict(torch.load(root+'/model/table2-3/'+stock+'_model_fractional_adam'+'_'+str(weight_decay)+'_'+str(lr)+'_'+str(alpha)+'_'+str(c)+'_.pth'))   #Fadam
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test_tensor)
        RMSE_test = RMSE(y_test_tensor.cpu().numpy(),test_outputs.cpu().detach().numpy())
        MAE_test = MAE(y_test_tensor.cpu().numpy(),test_outputs.cpu().detach().numpy())
        MAPE_test = MAPE(y_test_tensor.cpu().numpy(),test_outputs.cpu().detach().numpy())
        print(f'RMSE:{RMSE_test:.6f}')
        print(f'MAE:{MAE_test:.6f}')
        print(f'MAPE:{MAPE_test:.6f}')
        print(f'RMSE+MAE+MAPE:{RMSE_test+MAE_test+MAPE_test:.6f}')
        if best_evaluation > RMSE_test + MAE_test + MAPE_test:
            best_evaluation = RMSE_test + MAE_test + MAPE_test
            print('Best evaluation:',best_evaluation)
            best_alpha = alpha
            best_c = c
print('best_alpha:',best_alpha)
print('best_c:',best_c)
print('best_evaluation:',best_evaluation)



alpha: 0.9
c: 0.1
RMSE:0.419163
MAE:0.333128
MAPE:1.027757
RMSE+MAE+MAPE:1.780047
Best evaluation: 1.7800469

alpha: 0.9
c: 0.2
RMSE:0.533245
MAE:0.396797
MAPE:1.862432
RMSE+MAE+MAPE:2.792474

alpha: 0.9
c: 0.5
RMSE:0.487896
MAE:0.380897
MAPE:0.321037
RMSE+MAE+MAPE:1.189829
Best evaluation: 1.1898291

alpha: 0.9
c: 1.0
RMSE:0.413593
MAE:0.330468
MAPE:0.528094
RMSE+MAE+MAPE:1.272156

alpha: 0.9
c: 2.0
RMSE:0.468723
MAE:0.367907
MAPE:1.567707
RMSE+MAE+MAPE:2.404336

alpha: 0.9
c: 5.0
RMSE:0.434842
MAE:0.347378
MAPE:0.671503
RMSE+MAE+MAPE:1.453723

alpha: 0.9
c: 10.0
RMSE:0.459433
MAE:0.362773
MAPE:0.749048
RMSE+MAE+MAPE:1.571255

alpha: 0.8
c: 0.1
RMSE:0.468475
MAE:0.371608
MAPE:0.923165
RMSE+MAE+MAPE:1.763248

alpha: 0.8
c: 0.2
RMSE:0.370318
MAE:0.298048
MAPE:0.385427
RMSE+MAE+MAPE:1.053793
Best evaluation: 1.0537933

alpha: 0.8
c: 0.5
RMSE:0.424335
MAE:0.339415
MAPE:0.491378
RMSE+MAE+MAPE:1.255127

alpha: 0.8
c: 1.0
RMSE:0.401510
MAE:0.322265
MAPE:0.418126
RMSE+MAE+MAPE:1.141901

alph